In [40]:
import monai.transforms as mt
import logging
import sys
import matplotlib.pyplot as plt
import ignite
import numpy as np
import torch
import monai
import torchvision.transforms as transform
import warnings
warnings.filterwarnings("ignore")  # remove some scikit-image warnings

In [58]:
from segment_anything import SamPredictor, sam_model_registry 
import torch 
import os 
device = "cuda"
# 根据检查点加载模型
sam_model = sam_model_registry["vit_h"](checkpoint="./checkpoint/sam_vit_h_4b8939.pth")
sam_model.to(device=device)
sam_model.train()
predictor = SamPredictor(sam_model)

# 定义损失函数和优化器
optimizer = torch.optim.Adam(sam_model.mask_decoder.parameters()) 
criterion = torch.nn.BCEWithLogitsLoss()

In [72]:
def prompt_label(Ground_truth_mask,prompt = "box"):
    label_images = []
    prompt_images = []
    local_gt = Ground_truth_mask
    for i in range(1,np.max(local_gt)+1):
        i_gt_mask = np.where(local_gt == i, 1, 0)
        if np.sum(i_gt_mask.flatten()) == 0:
            continue
        else:
            label_images.append(i_gt_mask)
            if prompt == "point":
                # 随机选一个点
                indices = np.nonzero(i_gt_mask)
                random_index = np.random.randint(0,len(indices[0])-1)
                point = np.array([(indices[1][random_index],indices[0][random_index])])
                prompt_images.append(point)
                #SAM_mask , scores, logits = predictor.predict(point_coords=point,point_labels=np.array([1]),multimask_output=False,)
            elif prompt == "points":
                # 随机选5个点
                indices = np.nonzero(i_gt_mask)
                random_index = np.random.random_integers(0,len(indices[0])-1,5)
                points = []
                for j in range(len(random_index)):
                    points.append((indices[1][random_index[j]],indices[0][random_index[j]]))
                points = np.array(points)
                prompt_images.append(points)
                #SAM_mask , scores, logits = predictor.predict(point_coords=points,point_labels=np.ones(len(points)),multimask_output=False,)
            elif prompt == "box":
                indices = np.nonzero(i_gt_mask)
                x_min = min(indices[1])
                x_max = max(indices[1])
                y_min = min(indices[0])
                y_max = max(indices[0])
                input_box = np.array([x_min,y_min,x_max,y_max])
                prompt_images.append(input_box)
                #SAM_mask , scores, logits = predictor.predict(point_coords=None,point_labels=None,box=input_box,multimask_output=False,)
            else:
                raise NameError("prompt should be in [\"point\",\"points\",\"box\"]")
            
    return label_images, prompt_images

def Gray2RGB(image):
    input_img = np.zeros((image.shape[0],image.shape[1],3))
    input_img[:,:,0] = input_img[:,:,1] = input_img[:,:,2] = image[:,:] #(image-torch.min(image))/(torch.max(image))*255
    input_img = transform.ToTensor()(input_img).float()
    input_img = input_img.to(device="cuda")
    return input_img

In [43]:
from monai.apps import datasets
import json
import os
from monai import transforms
import torchvision
from monai.transforms import LoadImaged,EnsureChannelFirstd,Compose,ToTensord
import logging

image_transform = Compose([
    LoadImaged(keys=("image", "label")),
#    torchvision.transforms.Grayscale(num_output_channels=3),
    ToTensord(keys=("image", "label"))
])

with open("./data/dataset_0.json") as file1:
    dataset = json.load(file1)

#train_dataset =  monai.data.CacheDataset(dataset["training"], transform=image_transform)
train_dataset =  monai.data.Dataset(dataset["training"], transform=image_transform)
train_loader  = monai.data.DataLoader(train_dataset, batch_size=1, shuffle=True, num_workers=1, drop_last=True)

In [73]:
i = 0
for epoch in range(1):  # loop over the dataset multiple times
    running_loss = 0.0
    print(f"processing! {i*100 / 24}% finished")
    for data in train_loader:
        i += 1
        print(f"processing! {i*100 / 24}%")
        img,lab = data["image"][0],data["label"][0]
        train_img = []
        train_lab = []
        for i in range(img.shape[2]):
            if torch.sum(lab[:,:,i]) != 0:
                train_img.append(img[:,:,i])
                train_lab.append(lab[:,:,i])
        for j in range(len(train_img)):
            print(j)
            print(train_img[j].shape)
            optimizer.zero_grad()
            input_image = sam_model.preprocess(Gray2RGB(train_img[j]))
            print(input_image.shape)
            with torch.no_grad():
	            image_embedding = sam_model.image_encoder(input_image)
            loss = 0
            label_images, prompt_images = prompt_label(train_lab[j])
            for k in len(label_images):
                box_torch = torch.as_tensor(prompt_images[k], dtype=torch.int, device=device)
                sparse_embeddings, dense_embeddings = sam_model.prompt_encoder(points=None, boxes=box_torch,masks=None,)
                low_res_masks, iou_predictions = sam_model.mask_decoder(image_embeddings=image_embedding,image_pe=sam_model.prompt_encoder.get_dense_pe(),sparse_prompt_embeddings=sparse_embeddings,dense_prompt_embeddings=dense_embeddings,multimask_output=False,)
                #outputs = predictor.predict(point_coords=None,point_labels=None,box=prompt_images[k],multimask_output=False,)
                loss += criterion(outputs, label_images[k])
            loss.backward()
            optimizer.step()
        # load images with non zero label
print('Finished Training')

processing! 0.0% finished
processing! 4.166666666666667%
0
torch.Size([512, 512])
torch.Size([3, 1024, 1024])


RuntimeError: permute(sparse_coo): number of dimensions in the tensor input does not match the length of the desired ordering of dimensions i.e. input.dim() = 3 is not equal to len(dims) = 4

In [ ]:
PATH = "finetune/fine_tuned_sam.pth"
torch.save(sam_model.state_dict(), PATH)